In [2]:
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

df = pd.read_csv('CombinedTestResultMetric.csv')
df.drop(['participantCreatedOn', 'testResultMetricCreatedOn', 'testResultMetricTimestamp1', 'testResultMetricTimestamp2', 'Unnamed: 19'], axis = 1, inplace = True)

In [3]:
df['testStartedAt'] = df['testStartedAt'].apply(lambda x: pd.to_datetime(x[0:-10]))
df['testEndedAt'] = df['testEndedAt'].apply(lambda x: pd.to_datetime(x[0:-10]))
df['timecomplete'] = (df['testEndedAt'] - df['testStartedAt']).apply(lambda x: x.seconds)

In [4]:
df['age'] = [df.testStartedAt[i].year - df.participantBirthYear[i] 
                    for i in range(0, len(df))]
df.drop(['participantBirthYear', 'testStartedAt', 'testEndedAt'], axis = 1, inplace = True)

df['participantSex'].replace(['male', 'female'], [0, 1], inplace = True)

df['participantIsControl'].replace([True, False], [1, 0], inplace = True)

In [6]:
mean = df.groupby(['floodlightOpenId', 'testCode', 'testMetricCode']).mean()[['testResultMetricValue', 
                                                                              'timecomplete']]
mean.rename({'testResultMetricValue': 'val_mean', 'timecomplete': 'time_mean'}, axis = 1, inplace = True)
mean_reset = mean.reset_index()
mean_reset['test'] = mean_reset['testCode'] + mean_reset['testMetricCode']
meantest = mean_reset.drop(['testCode', 'testMetricCode'], axis = 1)
meantest = meantest.set_index(['floodlightOpenId', 'test'])
meantest = meantest.reset_index()
meantest = meantest.pivot(index='floodlightOpenId', columns='test', values=['val_mean', 'time_mean'])
meantest.columns = [col[0]+col[1] for col in meantest.columns]

In [9]:
var = df.groupby(['floodlightOpenId', 'testCode', 'testMetricCode']).var()[['testResultMetricValue', 
                                                                              'timecomplete']]
var.rename({'testResultMetricValue': 'val_var', 'timecomplete': 'time_var'}, axis = 1, inplace = True)
var_reset = var.reset_index()
var_reset['test'] = var_reset['testCode'] + var_reset['testMetricCode']
vartest = var_reset.drop(['testCode', 'testMetricCode'], axis = 1)
vartest = vartest.set_index(['floodlightOpenId', 'test'])
vartest = vartest.reset_index()
vartest = vartest.pivot(index='floodlightOpenId', columns='test', values=['val_var', 'time_var'])
vartest.columns = [col[0]+col[1] for col in vartest.columns]

In [23]:
count = df.groupby(['floodlightOpenId', 'testCode', 'testMetricCode']).count()[['testResultMetricValue']]
count.rename({'testResultMetricValue': 'test_count'}, axis = 1, inplace = True)
count_reset = count.reset_index()
count_reset['test'] = count_reset['testCode'] + count_reset['testMetricCode']
counttest = count_reset.drop(['testCode', 'testMetricCode'], axis = 1)
counttest = counttest.set_index(['floodlightOpenId', 'test'])
counttest = counttest.reset_index()
counttest = counttest.pivot(index='floodlightOpenId', columns='test', values='test_count')
counttest.columns = [col+'_count' for col in counttest.columns]

In [28]:
dfs = [meantest, vartest, counttest]
maindf = pd.concat(dfs, axis=1)

In [45]:
tests = df.groupby(['testCode', 'testMetricCode']).size().reset_index(name="Time")
subset = tests[['testCode', 'testMetricCode']]
testcombs = [tuple(x) for x in subset.values]
testcombs

[('daily_questions', 'mood_response'),
 ('draw_a_shape', 'bottom_to_top_hausdorff_distance_best'),
 ('draw_a_shape', 'circle_hausdorff_distance_best'),
 ('draw_a_shape', 'figure8_hausdorff_distance_best'),
 ('draw_a_shape', 'hand_used'),
 ('draw_a_shape', 'mean_hausdorff_distance_best'),
 ('draw_a_shape', 'number_shapes_drawn_correctly'),
 ('draw_a_shape', 'spiral_hausdorff_distance_best'),
 ('draw_a_shape', 'square_hausdorff_distance_best'),
 ('draw_a_shape', 'top_to_bottom_hausdorff_distance_best'),
 ('five_u_turn', 'turn_speed_avg'),
 ('five_u_turn', 'turns'),
 ('ips', 'correct_responses'),
 ('ips', 'response_time_avg'),
 ('ips_baseline', 'correct_responses'),
 ('ips_baseline', 'response_time_avg'),
 ('mobility', 'life_space_daily'),
 ('pinching', 'hand_used'),
 ('pinching', 'successful_pinches'),
 ('static_balance', 'sway_path'),
 ('two_min_walk', 'steps')]

In [47]:
names = ['mood', 'bottomtop', 'circle', 'figure8', 'shapehand', 'meandrawdist', 'numshapes', 'spiral', 'square',
        'topbottom', 'turnspeed', 'turns', 'ips', 'ipstime', 'bips', 'bipstime', 'mobility', 'pinchhand', 'pinches',
        'sway', 'steps']

In [53]:
def cols(names, metric):
    allcols = []
    for name in names: 
        allcols.append(name + '_' + metric)
    return allcols

In [86]:
meancols = cols(names, 'mean')
meantcols = cols(names, 'mean_t')
varcols = cols(names, 'var')
vartcols = cols(names, 'var_t')
countcols = cols(names, 'count')

In [88]:
allcols = meancols + meantcols + varcols + vartcols + countcols

In [90]:
maindf.columns = allcols

In [125]:
diagnosis = df.groupby(['floodlightOpenId', 'participantIsControl']).size().reset_index()[['floodlightOpenId', 'participantIsControl']]
sex = df.groupby(['floodlightOpenId', 'participantSex']).size().reset_index()[['floodlightOpenId', 'participantSex']]
country = df.groupby(['floodlightOpenId', 'participantCountryOfResidence']).size().reset_index(name='Time')[['floodlightOpenId', 'participantCountryOfResidence']]
age = df.groupby(['floodlightOpenId']).mean().reset_index()[['floodlightOpenId', 'age']]
weight = df.groupby(['floodlightOpenId']).mean().reset_index()[['floodlightOpenId', 'participantWeightLbs']]
height = df.groupby(['floodlightOpenId']).mean().reset_index()[['floodlightOpenId', 'participantHeightCms']]


In [196]:
maindf['sex'] = list(sex['participantSex'])
maindf['country'] = list(country['participantCountryOfResidence'])
maindf['age'] = list(age['age'])
maindf['weight'] = list(weight['participantWeightLbs'])
maindf['height'] = list(height['participantHeightCms'])
maindf['diagnosis'] = list(diagnosis['participantIsControl'])

In [198]:
maindf.head()

,mood_mean,bottomtop_mean,circle_mean,figure8_mean,shapehand_mean,meandrawdist_mean,numshapes_mean,spiral_mean,square_mean,topbottom_mean,...,pinchhand_count,pinches_count,sway_count,steps_count,sex,country,age,weight,height,diagnosis
floodlightOpenId,,,,,,,,,,,,,,,,,,,,,
FL00043910,3.2,0.435455,0.467273,0.463636,0.500000,0.438333,0.416667,0.496364,0.536364,0.455455,...,12.0,12.0,4.0,3.0,0,CA,0.0,91.0,100.0,1
FL00088674,4.0,0.130000,0.140000,0.120000,1.000000,0.150000,2.000000,0.170000,0.230000,0.140000,...,1.0,1.0,1.0,NaN,0,CA,38.0,245.0,188.0,0
FL00214638,NaN,20.000000,12.340000,12.340000,NaN,12.340000,NaN,12.340000,12.340000,12.340000,...,NaN,NaN,NaN,NaN,0,GB,29.0,150.0,160.0,0
FL00277926,3.0,0.136667,0.140000,0.123333,0.666667,0.156667,2.333333,0.170000,0.230000,0.140000,...,3.0,3.0,3.0,2.0,0,US,73.0,187.0,178.0,0
FL00497704,3.0,0.130000,0.120000,0.130000,1.000000,0.150000,1.000000,0.160000,0.220000,0.130000,...,1.0,1.0,1.0,NaN,1,US,30.0,130.0,171.0,0


In [208]:
maindf.iloc[:, 42:84]

,mood_var,bottomtop_var,circle_var,figure8_var,shapehand_var,meandrawdist_var,numshapes_var,spiral_var,square_var,topbottom_var,...,turns_var_t,ips_var_t,ipstime_var_t,bips_var_t,bipstime_var_t,mobility_var_t,pinchhand_var_t,pinches_var_t,sway_var_t,steps_var_t
floodlightOpenId,,,,,,,,,,,,,,,,,,,,,
FL00043910,0.600000,0.063307,0.041202,0.044645,0.272727,0.054542,1.356061,0.054205,0.019565,0.047427,...,0.000000,0.0,0.0,0.0,0.0,0.000000e+00,0.083333,0.083333,0.250000,0.000000
FL00088674,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.000000e+00,NaN,NaN,NaN,NaN
FL00214638,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
FL00277926,0.000000,0.000033,0.000400,0.000033,0.333333,0.000033,2.333333,0.000100,0.000700,0.000000,...,0.333333,NaN,NaN,NaN,NaN,0.000000e+00,0.000000,0.000000,0.333333,0.000000
FL00497704,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
FL00518623,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
FL00631304,0.285714,0.000114,0.000014,0.000014,0.285714,0.000048,3.285714,0.000148,0.000162,0.000024,...,0.238095,0.0,0.0,0.0,0.0,0.000000e+00,0.000000,0.000000,0.142857,NaN
FL00638936,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
FL00898809,0.500000,0.000000,0.000000,0.000200,0.500000,0.000050,0.000000,0.000000,0.000000,0.000000,...,0.000000,NaN,NaN,NaN,NaN,1.632343e+08,0.000000,0.000000,0.500000,NaN


In [282]:
# df[df['first_name'].notnull() & (df['nationality'] == "USA")]
# maindf[maindf['{}_var'.format('mood')].isna() & (maindf['{}_count'.format('mood')] == 1)].fillna(0)[['mood_var', 'mood_count']]

print(maindf['mood_var'].isna().sum())


def fillvar(df, param):
    return df[(df['{}_var'.format(param)].isna()) & (df['{}_count'.format(param)] ==1)].fillna(0, inplace = True)


446


In [304]:
maindf[maindf['{}_var'.format('mood')].isna() & (maindf['{}_count'.format('mood')] == 1)]

# if maindf['mood_count'] == 1:
#     maindf['mood_var'].fillna(1)
    
# while len(maindf[maindf['{}_var'.format('mood')].isna() & (maindf['{}_count'.format('mood')] == 1)]) > 0:
    
    
# for i in range(0, len(maindf)):
#     if maindf['mood_count'][i] == 1:
#         maindf['mood_var'][i].fillna(0)



,mood_mean,bottomtop_mean,circle_mean,figure8_mean,shapehand_mean,meandrawdist_mean,numshapes_mean,spiral_mean,square_mean,topbottom_mean,...,pinchhand_count,pinches_count,sway_count,steps_count,sex,country,age,weight,height,diagnosis
floodlightOpenId,,,,,,,,,,,,,,,,,,,,,
FL00088674,4.0,0.13,0.14,0.12,1.0,0.15,2.0,0.17,0.23,0.14,...,1.0,1.0,1.0,NaN,0,CA,38.0,245.0,188.0,0
FL00497704,3.0,0.13,0.12,0.13,1.0,0.15,1.0,0.16,0.22,0.13,...,1.0,1.0,1.0,NaN,1,US,30.0,130.0,171.0,0
FL00638936,4.0,0.13,0.14,0.11,1.0,0.14,2.0,0.16,0.18,0.14,...,1.0,1.0,1.0,NaN,1,US,49.0,140.0,152.0,1
FL00981454,4.0,0.21,0.11,0.11,1.0,0.17,3.0,0.16,0.21,0.19,...,1.0,1.0,1.0,NaN,0,US,29.0,165.0,183.0,1
FL01727090,5.0,NaN,NaN,NaN,1.0,0.00,0.0,NaN,NaN,NaN,...,1.0,1.0,1.0,1.0,1,US,30.0,128.0,175.0,0
FL01755190,3.0,0.12,0.12,0.11,1.0,0.15,2.0,0.17,0.22,0.14,...,1.0,1.0,1.0,NaN,0,IT,46.0,161.0,170.0,1
FL02122035,4.0,0.13,0.11,0.11,1.0,0.14,4.0,0.17,0.21,0.13,...,1.0,1.0,1.0,NaN,1,US,0.0,89.0,100.0,1
FL03317509,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,1,US,49.0,300.0,147.0,0
FL04217198,4.0,0.14,0.12,0.11,1.0,0.15,0.0,0.16,0.21,0.13,...,1.0,1.0,1.0,NaN,0,IT,30.0,159.0,179.0,1


In [202]:
pd.isnull(maindf).sum() > 500

mood_mean              False
bottomtop_mean         False
circle_mean            False
figure8_mean           False
shapehand_mean         False
meandrawdist_mean      False
numshapes_mean         False
spiral_mean            False
square_mean            False
topbottom_mean         False
turnspeed_mean         False
turns_mean             False
ips_mean               False
ipstime_mean           False
bips_mean              False
bipstime_mean          False
mobility_mean          False
pinchhand_mean         False
pinches_mean           False
sway_mean              False
steps_mean              True
mood_mean_t            False
bottomtop_mean_t       False
circle_mean_t          False
figure8_mean_t         False
shapehand_mean_t       False
meandrawdist_mean_t    False
numshapes_mean_t       False
spiral_mean_t          False
square_mean_t          False
                       ...  
pinches_var_t           True
sway_var_t              True
steps_var_t             True
mood_count    

In [211]:
for x in maindf['mood_var']:
    if x.isna():
        print(x)

AttributeError: 'float' object has no attribute 'isna'

In [306]:
maindf

,mood_mean,bottomtop_mean,circle_mean,figure8_mean,shapehand_mean,meandrawdist_mean,numshapes_mean,spiral_mean,square_mean,topbottom_mean,...,pinchhand_count,pinches_count,sway_count,steps_count,sex,country,age,weight,height,diagnosis
floodlightOpenId,,,,,,,,,,,,,,,,,,,,,
FL00043910,3.200000,0.435455,0.467273,0.463636,0.500000,0.438333,0.416667,0.496364,0.536364,0.455455,...,12.0,12.0,4.0,3.0,0,CA,0.000000,91.0,100.0,1
FL00088674,4.000000,0.130000,0.140000,0.120000,1.000000,0.150000,2.000000,0.170000,0.230000,0.140000,...,1.0,1.0,1.0,NaN,0,CA,38.000000,245.0,188.0,0
FL00214638,NaN,20.000000,12.340000,12.340000,NaN,12.340000,NaN,12.340000,12.340000,12.340000,...,NaN,NaN,NaN,NaN,0,GB,29.000000,150.0,160.0,0
FL00277926,3.000000,0.136667,0.140000,0.123333,0.666667,0.156667,2.333333,0.170000,0.230000,0.140000,...,3.0,3.0,3.0,2.0,0,US,73.000000,187.0,178.0,0
FL00497704,3.000000,0.130000,0.120000,0.130000,1.000000,0.150000,1.000000,0.160000,0.220000,0.130000,...,1.0,1.0,1.0,NaN,1,US,30.000000,130.0,171.0,0
FL00518623,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,1,CH,40.000000,133.0,177.0,0
FL00631304,4.571429,0.138571,0.111429,0.111429,0.571429,0.148571,4.428571,0.168571,0.214286,0.137143,...,7.0,7.0,7.0,1.0,1,CA,35.000000,131.0,167.0,0
FL00638936,4.000000,0.130000,0.140000,0.110000,1.000000,0.140000,2.000000,0.160000,0.180000,0.140000,...,1.0,1.0,1.0,NaN,1,US,49.000000,140.0,152.0,1
FL00898809,2.500000,0.140000,0.110000,0.120000,0.500000,0.145000,4.000000,0.160000,0.210000,0.140000,...,2.0,2.0,2.0,NaN,0,US,35.000000,179.0,178.0,1
